## 1. Setup Project Path

Configure the Python path to import project modules.

# Database Connection - Multi-Schema Access

This notebook demonstrates how to access tables across multiple database schemas (not just `dbo`).

## Key Topics:
1. Standard SQLDatabaseToolkit (dbo only limitation)
2. Querying all tables across all schemas
3. Listing all schemas in the database
4. Testing cross-schema queries
5. Creating custom tools for multi-schema access
6. Building agent-ready tools

This is important for databases with schema-qualified table names like `Sales.Customer`, `HumanResources.Employee`, etc.

In [ ]:
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)


## 2. Test Multi-Schema Database Access

This section tests various ways to access tables across different database schemas.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.tools import tool
from src.db.db_client import db_client
from src.config.models import get_llm, get_vector_embeddings
# Initialize model and database connection
# Note: No schema parameter = all schemas accessible via fully-qualified names
model = get_llm()
db = SQLDatabase.from_uri(db_client.get_connection_uri())

# === 1. Standard toolkit list_tables (dbo only) ===
# The default toolkit only shows tables from the 'dbo' schema
print("=== 1. Standard toolkit list_tables (dbo only) ===")
toolkit = SQLDatabaseToolkit(db=db, llm=model)
tools = toolkit.get_tools()
tables_tool = next(t for t in tools if t.name == "sql_db_list_tables")
print(tables_tool.invoke({}))

# === 2. ALL tables across ALL schemas ===
# Query INFORMATION_SCHEMA to get tables from all schemas
print("\n=== 2. ALL tables across ALL schemas ===")
all_tables = db.run("""
    SELECT TABLE_SCHEMA, TABLE_NAME, TABLE_TYPE
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE'
    ORDER BY TABLE_SCHEMA, TABLE_NAME
""")
# Truncate output if too long for readability
print(all_tables[:500] + "..." if len(all_tables) > 500 else all_tables)

# === 3. ALL schemas ===
# Get distinct list of all schemas in the database
print("\n=== 3. ALL schemas ===")
schemas = db.run("""
    SELECT DISTINCT TABLE_SCHEMA 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE'
    ORDER BY TABLE_SCHEMA
""")
print(schemas)

# === 4. Test cross-schema query ===
# Test querying a table from a non-dbo schema (e.g., Sales)
print("\n=== 4. Test cross-schema query ===")
try:
    # Example: Query from Sales schema using fully-qualified name
    sample_query = "SELECT TOP 3 * FROM Sales.SalesOrderDetail"
    result = db.run(sample_query)
    print("Sales schema works:", result[:200] + "..." if len(result) > 200 else result)
except Exception as e:
    print(f"Error: {e}")
    print("Note: Replace with an actual table name from step 2")

# === 5. Custom tool for agent (all schemas) ===
# Create a custom tool that lists ALL tables with schema prefixes
print("\n=== 5. Custom tool for agent (all schemas) ===")
@tool
def list_all_tables() -> str:
    """List ALL tables across ALL schemas (not just dbo).
    Returns fully-qualified table names in format: schema.table_name"""
    return db.run("""
        SELECT TABLE_SCHEMA + '.' + TABLE_NAME as full_table_name
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_TYPE = 'BASE TABLE'
        ORDER BY TABLE_SCHEMA, TABLE_NAME
    """)

print(list_all_tables.invoke({}))

# === 6. Full agent-ready tools ===
# Combine standard toolkit tools with our custom multi-schema tool
print("\n=== 6. Full agent-ready tools ===")
agent_tools = toolkit.get_tools() + [list_all_tables]
for tool in agent_tools:
    # Print each tool name and description (truncated for readability)
    print(f"- {tool.name}: {tool.description[:100]}...")

=== 1. Standard toolkit list_tables (dbo only) ===
AWBuildVersion, DatabaseLog, ErrorLog

=== 2. ALL tables across ALL schemas ===
[('dbo', 'AWBuildVersion', 'BASE TABLE'), ('dbo', 'DatabaseLog', 'BASE TABLE'), ('dbo', 'ErrorLog', 'BASE TABLE'), ('HumanResources', 'Department', 'BASE TABLE'), ('HumanResources', 'Employee', 'BASE TABLE'), ('HumanResources', 'EmployeeDepartmentHistory', 'BASE TABLE'), ('HumanResources', 'EmployeePayHistory', 'BASE TABLE'), ('HumanResources', 'JobCandidate', 'BASE TABLE'), ('HumanResources', 'Shift', 'BASE TABLE'), ('Person', 'Address', 'BASE TABLE'), ('Person', 'AddressType', 'BASE TABLE'), (...

=== 3. ALL schemas ===
[('dbo',), ('HumanResources',), ('Person',), ('Production',), ('Purchasing',), ('Sales',)]

=== 4. Test cross-schema query ===
Sales schema works: [(43659, 1, '4911-403C-98', 1, 776, 1, Decimal('2024.9940'), Decimal('0.0000'), Decimal('2024.994000'), 'B207C96D-D9E6-402B-8470-2CC176C42283', datetime.datetime(2011, 5, 31, 0, 0)), (43659, 2, 

c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'DatabaseUser'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Event'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Schema'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Object'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain